# NonStaticCNN_모델 만들기.ipynb

**1.FastText를 사용한 초기 임베딩 레이어**
- 임베딩 레이어; 단어간의 의미 유사도를 보여주는 행렬
- 기존에는 초기 임베딩 레이어는 0으로 하고, 영화리뷰 90%, 맛집리뷰 10%의 비율로 임베딩 레이어가 생성되므로 영화도메인, 맛집도메인에서 차이가 클 때 긍부정 분류가 잘 안될 수 있음.
- FastText에서 사전 학습된 초기 임베딩 레이어로 시작.
  - FastText는 서브워드 기반으로 단어 OOV 문제를 완화하며, 비격식적 말투에서도 기본적인 성능을 제공.
  -[FastText 다운로드링크](https://github.com/facebookresearch/fastText/blob/master/docs/crawl-vectors.md) 아래로 내려가면 한국어에서 `cc.ko.300.vec.gz` 파일을 받으면 된다.


**2. 문장 분리**
- kkoma(꼬꼬마)를 사용하면 문장부호 없이도 문장을 분리할 수 있다고 함.
- 하나의 리뷰에 여러개의 감정이 섞여있을 수 있어서 kkoma를 사용한 문장분리 추가.
- FastText는 서브워드 기반이므로 문법 교정의 필요성이 낮아지므로 문법 교정은 생략


**3. 초기 학습 데이터**
- AIhub에서 제공되는 한국어 단발성 일반 담화 데이터로 대체함. [링크](https://aihub.or.kr/aihubdata/data/view.do?dataSetSn=270)
- 도메인에 상관없이 모든 상황에서 기초로 적용할 수 있는 담화 데이터임.


### Non Static CNN
우리 프로젝트에서 적합한 이유
- 로컬 특징을 잘 추출해서 짧은 텍스트에 적합함: CNN은 다양한 크기의 필터를 사용해 텍스트 내의 로컬 패턴(2~4단어 수준의 n-그램)을 잘 학습.
- 임베딩을 업데이트할 수 있음: 사전 학습된 임베딩에서 시작해서 특정 도메인에 특화된 임베딩으로 수정이 가능함. 맛집 특화된 임베딩이 없는 현 상황에서 최선의 선택지일 것 같음.


In [ ]:
#'세션 다시 시작'이라는 안내메시지가 나오는 경우, "취소"하면 됨. (설치는 문제없이 되는듯??)
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install konlpy
!pip install --upgrade tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import io
import pickle
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Concatenate, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Kkma
from pykospacing import Spacing
from hanspell import spell_checker

import os
os.environ['JAVA_TOOL_OPTIONS'] = "-Xmx4g -Xms512m" #텍스트 전처리에서 과부하 문제가 있어서 방지용

In [ ]:
# 1. FastText 임베딩 로드
def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    n, d = map(int, fin.readline().split())  # 첫 줄: 단어 수와 차원 수 읽기
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')  # 공백 기준으로 단어와 벡터 분리
        word = tokens[0]  # 단어
        vector = list(map(float, tokens[1:]))  # 벡터값 (실수로 변환)
        data[word] = vector  # 딕셔너리에 저장
    return data

In [ ]:
# 2. 임베딩 행렬 생성
def create_embedding_matrix(tokenizer, embedding_dict, vocab_size, embedding_dim):
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in tokenizer.word_index.items():
        if i >= vocab_size:
            continue
        vector = embedding_dict.get(word)
        if vector is not None:
            embedding_matrix[i] = vector
    print(f"Embedding matrix shape: {embedding_matrix.shape}")
    return embedding_matrix

In [ ]:
# 3. 비정태적 CNN 모델 정의 (참고논문과 같은 모델으로 정의함)
def build_nonstatic_cnn(vocab_size, embedding_dim, embedding_matrix, max_len):
    # 1. 입력 계층 정의
    input_layer = Input(shape=(max_len,), dtype='int32')

    # 2. 임베딩 레이어: 비정태적 (사전 학습된 임베딩에서 시작, 학습 가능)
    embedding_layer = Embedding(input_dim =vocab_size,
                                 output_dim=embedding_dim,
                                 weights=[embedding_matrix],
                                 trainable=True)(input_layer) #임베딩 레이어를 학습 가능한 상태로 설정

    # 3. 합성곱 계층: 다양한 필터 크기 사용 (2, 3, 4, 5)
    conv_2 = Conv1D(filters=50, kernel_size=2, activation='relu', padding='same')(embedding_layer)
    conv_3 = Conv1D(filters=50, kernel_size=3, activation='relu', padding='same')(embedding_layer)
    conv_4 = Conv1D(filters=50, kernel_size=4, activation='relu', padding='same')(embedding_layer)

    # 4. 맥스풀링 계층: Global MaxPooling 사용
    pool_2 = GlobalMaxPooling1D()(conv_2)
    pool_3 = GlobalMaxPooling1D()(conv_3)
    pool_4 = GlobalMaxPooling1D()(conv_4)
    # Global MaxPooling은 가장 중요한 특정 패턴만 남기고 결과물의 크기를 축소하여 1x50으로 통일함.
    # 예를 들어, 리뷰가 blah blah 정말 맛있어요 blah blah라면, 다른 패턴보다 '정말 맛있어요'에 집중

    # 5. 출력 연결: 여러 맥스풀링 결과를 Concatenate
    concatenated = Concatenate()([pool_2, pool_3, pool_4])

    # 6. Fully Connected Layer 1: 은닉층 (50 units)
    dense_1 = Dense(50, activation='relu')(concatenated)
    dropout_1 = Dropout(0.5)(dense_1)

    # 7. Fully Connected Layer 2: 출력층 (이진 분류 - Sigmoid)
    output_layer = Dense(1, activation='sigmoid')(dropout_1)

    # 8. 모델 생성
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# 4. 데이터 전처리

# 꼬꼬마 문장 분리 객체 생성
kkma = Kkma()

# 1. 전처리 함수: 특수문자 제거, 공백 처리, 맞춤법 교정
def preprocess_text(data):
    #데이터프레임 형식이 아닌 경우 오류남
    if isinstance(data, (list, np.ndarray)):
        data = pd.DataFrame(data, columns=['reviews'])
    elif not isinstance(data, pd.DataFrame):
        raise ValueError("입력 데이터는 DataFrame, list, 또는 ndarray 형태여야 합니다.")

    # 한글, 숫자, 공백, 언더스코어 외의 모든 문자 제거 (\n 포함)
    data['reviews'] = data['reviews'].str.replace("[^\w\sㄱ-ㅎㅏ-ㅣ가-힣]", " ", regex=True)

    # 맞춤법 및 띄어쓰기 교정
    #data['reviews'] = data['reviews'].apply(lambda x: spacing(x) if isinstance(x, str) else x)

    # 공백만 남은 경우 NaN 처리
    data['reviews'].replace('', np.nan, inplace=True)

    #전처리 후 남은 글자가 2글자 미만이라면 삭제함
    data['reviews'] = data['reviews'].apply(lambda x: np.nan if isinstance(x,str) and len(x.strip()) <= 5 else x)

    # NaN 제거
    data.dropna(subset=['reviews'], inplace=True)

    return data

# 2. 문장 분리 함수: 꼬꼬마를 활용해 문장 단위로 분리
def split_sentences(data):
    if isinstance(data, list) or isinstance(data, np.ndarray):
        data = pd.DataFrame(data, columns=['reviews'])
    data['reviews'] = data['reviews'].apply(lambda x: kkma.sentences(x) if isinstance(x, str) else [])
    data = data.explode('reviews').reset_index(drop=True)
    return data

# 3. 최종 데이터 전처리 함수
def preprocess_data(data, tokenizer, max_len):
    # 기존 전처리 적용
    data = preprocess_text(data)
    # 문장 분리 적용
    data = split_sentences(data)
    # 토큰화 및 패딩
    return data

------

In [ ]:
# ModelCheckpoint 설정 (최고 성능 모델만 저장)
checkpoint = ModelCheckpoint("/content/drive/My Drive/24-2Main/best_restaurant_sentiment_model.keras",
                             monitor='val_loss',
                             save_best_only=True,
                             verbose=1)

In [ ]:
# FastText 딕셔너리 불러오기
with open("/content/drive/MyDrive/24-2Main/data/fasttext_embedding.pkl", "rb") as f:
    embedding_dict = pickle.load(f)

In [ ]:
# 2. 데이터 로드 및 초기 전처리
from sklearn.utils import resample
# 한국어 단발성 대화 데이터로 초기 학습
labeled_data = pd.read_csv("/content/drive/MyDrive/24-2Main/data/한국어단발성데이터셋.csv", index_col=False)  # 긍/부정 라벨 데이터
labeled_data.rename(columns={"Sentence": "reviews", "Emotion":"label"}, inplace=True) # 함수 사용하기 위해서 칼럼명 맞춰드림

# 클래스별 데이터 분리
negative = labeled_data[labeled_data['label'] == 0]
positive = labeled_data[labeled_data['label'] == 1]

# 긍정 데이터 다운샘플링
positive_downsampled = resample(positive,
                                replace=False,  # 복원 샘플링 여부
                                n_samples=4000,  # 4,000개로 샘플링
                                random_state=42)

# 부정 데이터 다운샘플링
negative_downsampled = resample(negative,
                                replace=False,  # 복원 샘플링 여부
                                n_samples=4000,  # 4,000개로 샘플링
                                random_state=42)

# 다운샘플링 데이터 결합
balanced_data = pd.concat([negative_downsampled, positive_downsampled])

# 데이터 셔플링
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

balanced_data

In [ ]:
#토크나이저 정의 및 학습
tokenizer = Tokenizer()
tokenizer.fit_on_texts(balanced_data['reviews'])
vocab_size = len(tokenizer.word_index) + 1
max_len = 80  # 리뷰 최대 길이

with open("/content/drive/MyDrive/24-2Main/data/tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# 임베딩 행렬 생성
embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1
embedding_matrix = create_embedding_matrix(tokenizer, embedding_dict, vocab_size, embedding_dim)

Embedding matrix shape: (24428, 300)


In [ ]:
print("1", balanced_data)
preprocessed_df = preprocess_data(balanced_data, tokenizer, max_len)
print("2", preprocessed_df)

X = preprocessed_df['reviews']
y = preprocessed_df['label']
X.to_csv("/content/drive/MyDrive/24-2Main/data/X.csv", index=False)
y.to_csv("/content/drive/MyDrive/24-2Main/data/y.csv", index=False)

sequences = tokenizer.texts_to_sequences(X if isinstance(X, list) else X)
X = pad_sequences(sequences, maxlen=max_len)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# 4. 비정태적 CNN 모델 학습
# 모델 빌드
model = build_nonstatic_cnn(vocab_size, embedding_dim, embedding_matrix, max_len)

# Early Stopping 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 학습
model.fit(X_train, y_train,
          epochs=10,
          batch_size=64,
          validation_data=(X_val, y_val),
          callbacks=[early_stopping, checkpoint])